### Vector Stores and retrievers

This tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data -- from (vector) databases and other sources -- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

We will cover:
- Documents
- Vector Stores
- Retrievers

### Documents

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has thwo attributes:

- page_content: a string representing the content.
- metadata: a dict containing arbitrary metadata. The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that and individual Document object often represents a chunk of a larger document.
- id: (optional) a string identifier for the document.

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY") # type: ignore
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN") # type: ignore

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x13eb98b30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x13ebb4e90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

In [6]:
## VectorStores
from langchain_chroma import Chroma

vectorStore = Chroma.from_documents(documents, embedding=embeddings)
vectorStore



In [7]:
vectorStore.similarity_search("cat")

[Document(id='c5b0abdb-dcdf-4e87-97dd-85ccc3b112bc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b537dd87-ce20-4bef-b85c-d137925e8620', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [8]:
## Async query
await vectorStore.asimilarity_search("cat")

[Document(id='c5b0abdb-dcdf-4e87-97dd-85ccc3b112bc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b537dd87-ce20-4bef-b85c-d137925e8620', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [9]:
vectorStore.similarity_search_with_score("cat")

[(Document(id='c5b0abdb-dcdf-4e87-97dd-85ccc3b112bc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351055026054382),
 (Document(id='b537dd87-ce20-4bef-b85c-d137925e8620', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706)]

### Retrievers

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implemente a standard set of methods (e.g., synchronous and asynchronous invike and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourserlves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [11]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# The similarity_search function is going to get applied to all the parameters that we are giving inside the batch function
retriever = RunnableLambda(vectorStore.similarity_search).bind(k=1)
retriever.batch(["cat", "dog"])

[[Document(id='c5b0abdb-dcdf-4e87-97dd-85ccc3b112bc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='b537dd87-ce20-4bef-b85c-d137925e8620', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

VectorStores implemen an as_retriever method that will generate a Retreiver, specifically a VectorStoreRetriever. These retrievers include specific search_type and search-.kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [12]:
retriever = vectorStore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 1}
)

retriever.batch(["cat", "dog"])

[[Document(id='c5b0abdb-dcdf-4e87-97dd-85ccc3b112bc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='b537dd87-ce20-4bef-b85c-d137925e8620', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
## RAG 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context": retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dogs")
response

AIMessage(content='Dogs are great companions, known for their loyalty and friendliness.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 104, 'total_tokens': 120, 'completion_time': 0.014595043, 'completion_tokens_details': None, 'prompt_time': 0.006866647, 'prompt_tokens_details': None, 'queue_time': 0.005514752, 'total_time': 0.02146169}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--df51fd8c-5fe2-4196-af45-b1355f8e7c76-0', usage_metadata={'input_tokens': 104, 'output_tokens': 16, 'total_tokens': 120})